In [17]:
# Skin Cancer Predictor

data_dir = './'

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import json
import os
from torchvision.io import read_image
from PIL import Image
import yaml

import yaml
import numpy as np   # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets,models
import math
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
import time
from PIL import Image
from datetime import datetime

# Get configs from config file
stream = open("config.yaml", 'r')
config_dict = yaml.safe_load(stream)
batch_size = config_dict['batch_size']
learning_rate = config_dict['lr']
model_pth = config_dict['model_pth']
train_data = config_dict['train_data']
valid_data = config_dict['valid_data']
test_data = config_dict['test_data']

In [18]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((230, 230)),
        transforms.RandomRotation(30,),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'valid': transforms.Compose([
        transforms.Resize((400, 400)),
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [19]:
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

In [20]:
# Class for loading test dataset
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        img_path = self.path
        image = pil_loader(img_path)
        if self.transform:
            image = self.transform(image)
        
        return image, 0

In [33]:
# 

def idx_to_class(indices):
    indices = indices[0]
    classes  = []
    for index in indices:
        
        if index==0:
            classes.append('benign')
        else:
            classes.append("malignant")
    return classes

In [25]:


class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)



In [27]:
def predict(model_path, dataloader, print_progress=False):

    """

    :param model_path: Path of Model used for prediction
    :param dataloader: Test DataLoader
    :param print_progress: Prints progress if True
    :return: Prediction(as a list) on test folder defined by config file
    """

    model = torch.load(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()

    predictions = {}
    with torch.no_grad():
        for ii, (images,_) in enumerate(dataloader, start=1):

            if print_progress:
                if ii % 5 == 0:
                    print('Batch {}/{}'.format(ii, len(dataloader)))
            
            images = images.to(device)
            
            logps = model(images)
            ps = torch.exp(logps)

            # Top indices
            _, top_indices = ps.topk(1)
            top_indices = top_indices.detach().cpu().numpy().tolist()
            #print(top_indices)
            # Convert indices to classes
            top_classes = idx_to_class(top_indices)
            # print("Img:" ,img_names)
            #for i, img_name in enumerate(img_names):
                #predictions[img_name] = top_classes[i]

        #print('\nPrediction Generation Completed')

    return top_classes


In [28]:
#model_path = data_dir + os.path.join(model_pth,'dense121.h5')
model_path = "D:\Xview2\Jeevan_Data_Hackfest'22\Skin_Cancer\Models\85acc.pth"
test_dataset = TestDataset("./100.jpg",data_transforms['test'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

pre = predict(model_path,test_loader)

c:\users\av192\appdata\local\programs\python\python39\lib\site-packages\torch\serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [29]:
pre

['Mal']

In [31]:
import joblib
kl = joblib.load('./Models/class.pkl')

In [23]:
os.listdir('./')

['.ipynb_checkpoints', '100.jpg', 'config.yaml', 'Models', 'Predict_1.ipynb']

In [32]:
kl

{'benign': 0, 'malignant': 1}